In [ ]:
from diffusers import StableDiffusionXLInpaintPipeline, AutoencoderKL, StableDiffusionXLPipeline, StableDiffusionXLImg2ImgPipeline, ControlNetModel, StableDiffusionXLControlNetImg2ImgPipeline
import torch
from PIL import Image, ImageOps
import numpy as np
import cv2
import os

In [ ]:
vae = AutoencoderKL.from_pretrained("stabilityai/sdxl-vae", torch_dtype=torch.float16)
pipe = StableDiffusionXLPipeline.from_single_file("/home/oleksandr/projects/upwork/LoDi-Engine-Runpod/models_hotswap/juggernautXL_v7Rundiffusion.10002.safetensors", torch_dtype=torch.float16).to(torch.device('cuda'))
pipe_img2img = StableDiffusionXLImg2ImgPipeline(**pipe.components)
controlnet_cn = ControlNetModel.from_pretrained(
        "diffusers/controlnet-canny-sdxl-1.0", 
        torch_dtype=torch.float16, 
        variant="fp16",
        use_safetensors=True,
    ).to("cuda")
pipe_cn = StableDiffusionXLControlNetImg2ImgPipeline(**pipe.components, controlnet=controlnet_cn).to("cuda")

In [ ]:
qrimg = Image.open("/home/oleksandr/projects/upwork/esov-api/tmp/qr/Piece 01.png")
style_fold = "/home/oleksandr/projects/upwork/esov-api/tmp/qr/Reference - General Style"
img_fold = "/home/oleksandr/projects/upwork/esov-api/tmp/qr/Style Scene 001"

style_imgs = []
for file in os.listdir(style_fold):
    style_img = Image.open(os.path.join(style_fold, file)).convert("RGB")
    style_imgs.append(style_img)

imgs = []
for file in os.listdir(img_fold):
    img = Image.open(os.path.join(img_fold, file)).convert("RGB")
    imgs.append(img)

print(f"QR Image size: {qrimg.size}")
print(f"Style Image size: {style_imgs[0].size}")
print(f"Image size: {imgs[0].size}")

In [ ]:
ImageOps.invert(qrimg.convert("L")).convert("L")

In [ ]:
# overlay img[0] and qrimg with qrimg mask and 0.5 blend
qr_img_resized = qrimg.resize(imgs[0].size).convert("RGBA")
to_blend = imgs[0].copy().convert("RGBA")
blended = Image.blend(to_blend, qr_img_resized, 0.7)
pasted = imgs[0].copy()
pasted.paste(blended, (0, 0), ImageOps.invert(qrimg.convert("L")).resize(imgs[0].size))
pasted

In [ ]:
def getCannyCondImage(pilimg, low_threshold=100, high_threshold=200):
    img = np.array(pilimg)
    edges = cv2.Canny(img, low_threshold, high_threshold)
    return Image.fromarray(edges)
condimg = getCannyCondImage(qrimg)
condimg

In [ ]:
w=1024
h=1024

generator = torch.Generator(device="cuda").manual_seed(1)
pipe_cn(
    "image with bars over it",
    negative_prompt="",
    image=imgs[0].resize((w, h)),
    control_image = condimg.resize((w, h)),
    strength=0.4,
    guidance_scale=20,
    num_inference_steps=60,
    generator=generator,
    w=w,
    h=h,
    controlnet_conditioning_scale=3.0,
).images[0]